# 신용카드 고객 세그먼트 분류 AI 경진대회

## 1. 데이터 개요 및 설명

본 노트북은 [DACON 신용카드 고객 세그먼트 분류 AI 경진대회](https://dacon.io/competitions/official/236460/data) 데이터를 활용하여 고객의 신용 등급(Credit)을 예측하는 모델을 개발하는 과정을 담고 있습니다.

### 목표
- 고객의 개인 정보(소득, 직업, 가족 구성 등)와 금융 이력 데이터를 분석
- 사용자별 신용 등급을 0, 1, 2 세 가지 클래스로 분류

### 데이터셋 구성
- **train.csv**: 학습용 데이터 (신용 등급 포함)
- **test.csv**: 예측용 테스트 데이터 (신용 등급 미포함)
- **sample_submission.csv**: 제출 양식

주요 컬럼:
- `gender`, `car`, `reality`: 성별, 차 소유, 부동산 소유
- `income_total`: 연간 소득
- `income_type`, `edu_type`, `family_type`, `house_type`: 소득 분류, 교육 수준, 가족 형태, 주거 형태
- `DAYS_BIRTH`, `DAYS_EMPLOYED`: 출생일, 고용일 (음수 값, 0은 오늘을 의미)
- `begin_month`: 신용카드 발급 월 (음수)
- `occyp_type`: 직업 유형
- `credit`: 신용 등급 (Target, 0/1/2)

### 1. 환경세팅

In [3]:
pip install pandas pyarrow

Note: you may need to restart the kernel to use updated packages.


In [9]:
# 환경 세팅
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### 2. parquet 파일 읽고 데이터 확인

In [5]:
import pandas as pd

# 파일 읽기
df = pd.read_parquet(r'C:\Users\Brain\github\DataSicence\Credit Card\data\train\1.회원정보\201807_train_회원정보.parquet')

# 상위 5개 데이터 확인
print(df.head())

     기준년월            ID  남녀구분코드   연령 Segment  회원여부_이용가능  회원여부_이용가능_CA  \
0  201807  TRAIN_000000       2  40대       D          1             1   
1  201807  TRAIN_000001       1  30대       E          1             1   
2  201807  TRAIN_000002       1  30대       C          1             1   
3  201807  TRAIN_000003       2  40대       D          1             1   
4  201807  TRAIN_000004       2  40대       E          1             1   

   회원여부_이용가능_카드론  소지여부_신용  소지카드수_유효_신용  ...  할인금액_제휴연회비_B0M  청구금액_기본연회비_B0M  \
0              0        1            1  ...               0               0   
1              1        1            1  ...               0               0   
2              0        1            1  ...               0               0   
3              0        1            2  ...               0               0   
4              1        1            1  ...               0               0   

   청구금액_제휴연회비_B0M  상품관련면제카드수_B0M  임직원면제카드수_B0M  우수회원면제카드수_B0M  기타면제카드수_B0M  \
0       

In [ ]:
# 하위 5개 데이터 확인
print(df.tail())

          기준년월            ID  남녀구분코드     연령 Segment  회원여부_이용가능  회원여부_이용가능_CA  \
399995  201807  TRAIN_399995       2  70대이상       E          1             1   
399996  201807  TRAIN_399996       2    50대       D          1             1   
399997  201807  TRAIN_399997       1    30대       C          1             1   
399998  201807  TRAIN_399998       1    40대       E          1             1   
399999  201807  TRAIN_399999       2    40대       E          1             1   

        회원여부_이용가능_카드론  소지여부_신용  소지카드수_유효_신용  ...  할인금액_제휴연회비_B0M  \
399995              1        1            1  ...               0   
399996              1        1            1  ...               0   
399997              0        1            1  ...               0   
399998              1        1            1  ...               0   
399999              0        1            1  ...               0   

        청구금액_기본연회비_B0M  청구금액_제휴연회비_B0M  상품관련면제카드수_B0M  임직원면제카드수_B0M  \
399995               0               0 

In [26]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 78 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   기준년월                400000 non-null  int64  
 1   ID                  400000 non-null  object 
 2   남녀구분코드              400000 non-null  int64  
 3   연령                  400000 non-null  object 
 4   Segment             400000 non-null  object 
 5   회원여부_이용가능           400000 non-null  int64  
 6   회원여부_이용가능_CA        400000 non-null  int64  
 7   회원여부_이용가능_카드론       400000 non-null  int64  
 8   소지여부_신용             400000 non-null  int64  
 9   소지카드수_유효_신용         400000 non-null  int64  
 10  소지카드수_이용가능_신용       400000 non-null  int64  
 11  입회일자_신용             400000 non-null  int64  
 12  입회경과개월수_신용          400000 non-null  int64  
 13  회원여부_연체             400000 non-null  int64  
 14  이용거절여부_카드론          400000 non-null  int64  
 15  동의여부_한도증액안내         400000 non-nul

### 3. 파일 불러오기

#### train 데이터 생성

In [10]:
# 각 폴더명 딕셔너리
folder_name = {1:"1.회원정보", 2:"2.신용정보", 3:"3.승인매출정보", 4:"4.청구입금정보", 5:"5.잔액정보", 6:"6.채널정보", 7:"7.마케팅정보", 8:"8.성과정보"}

# 현재 디렉터리
HOME = os.getcwd()

# 폴더 리스트를 생성하고 폴더 경로들을 리스트로 저장하기
folder_list = []
for value in folder_name.values():
    folder_list.append(os.path.join(HOME, "data", "train", value))
print(len(folder_list))

# 폴더 내 파일 리스트 읽어오기
fileNameList = {}
for i in range(len(folder_list)):
    fileNameList[i+1] = os.listdir(folder_list[i])
print(fileNameList)

# 파일 경로 리스트 생성
filePathList = {}
temp = []
for i in range(len(folder_list)):
    for j in range(len(fileNameList[i+1])):
        temp.append(os.path.join(folder_list[i], fileNameList[i+1][j]))
    filePathList[i+1] = temp
    temp = []

print(filePathList)

8
{1: ['201807_train_회원정보.parquet', '201808_train_회원정보.parquet', '201809_train_회원정보.parquet', '201810_train_회원정보.parquet', '201811_train_회원정보.parquet', '201812_train_회원정보.parquet'], 2: ['201807_train_신용정보.parquet', '201808_train_신용정보.parquet', '201809_train_신용정보.parquet', '201810_train_신용정보.parquet', '201811_train_신용정보.parquet', '201812_train_신용정보.parquet'], 3: ['201807_train_승인매출정보.parquet', '201808_train_승인매출정보.parquet', '201809_train_승인매출정보.parquet', '201810_train_승인매출정보.parquet', '201811_train_승인매출정보.parquet', '201812_train_승인매출정보.parquet'], 4: ['201807_train_청구정보.parquet', '201808_train_청구정보.parquet', '201809_train_청구정보.parquet', '201810_train_청구정보.parquet', '201811_train_청구정보.parquet', '201812_train_청구정보.parquet'], 5: ['201807_train_잔액정보.parquet', '201808_train_잔액정보.parquet', '201809_train_잔액정보.parquet', '201810_train_잔액정보.parquet', '201811_train_잔액정보.parquet', '201812_train_잔액정보.parquet'], 6: ['201807_train_채널정보.parquet', '201808_train_채널정보.parquet', '201809_train_채널정보.parquet',

In [12]:
# 파일 딕셔너리 생성
train_List = {}
temp = []
i = 0
# 파일 리스트에 각 파일의 데이터 프레임 저장
for files in filePathList.values():
    for file in files:
        temp.append(pd.read_parquet(file, engine='fastparquet'))
    i = i+1
    train_List[i] = temp
    temp = []

### test 데이터 생성

In [13]:
# 각 폴더명 딕셔너리
folder_name = {1:"1.회원정보", 2:"2.신용정보", 3:"3.승인매출정보", 4:"4.청구입금정보", 5:"5.잔액정보", 6:"6.채널정보", 7:"7.마케팅정보", 8:"8.성과정보"}

# 현재 디렉터리
HOME = os.getcwd()

# 폴더 리스트를 생성하고 폴더 경로들을 리스트로 저장하기
folder_list = []
for value in folder_name.values():
    folder_list.append(os.path.join(HOME, "data", "test", value))
print(len(folder_list))

# 폴더 내 파일 리스트 읽어오기
fileNameList = {}
for i in range(len(folder_list)):
    fileNameList[i+1] = os.listdir(folder_list[i])
print(fileNameList)

# 파일 경로 리스트 생성
filePathList = {}
temp = []
for i in range(len(folder_list)):
    for j in range(len(fileNameList[i+1])):
        temp.append(os.path.join(folder_list[i], fileNameList[i+1][j]))
    filePathList[i+1] = temp
    temp = []

print(filePathList)

8
{1: ['201807_test_회원정보.parquet', '201808_test_회원정보.parquet', '201809_test_회원정보.parquet', '201810_test_회원정보.parquet', '201811_test_회원정보.parquet', '201812_test_회원정보.parquet'], 2: ['201807_test_신용정보.parquet', '201808_test_신용정보.parquet', '201809_test_신용정보.parquet', '201810_test_신용정보.parquet', '201811_test_신용정보.parquet', '201812_test_신용정보.parquet'], 3: ['201807_test_승인매출정보.parquet', '201808_test_승인매출정보.parquet', '201809_test_승인매출정보.parquet', '201810_test_승인매출정보.parquet', '201811_test_승인매출정보.parquet', '201812_test_승인매출정보.parquet'], 4: ['201807_test_청구정보.parquet', '201808_test_청구정보.parquet', '201809_test_청구정보.parquet', '201810_test_청구정보.parquet', '201811_test_청구정보.parquet', '201812_test_청구정보.parquet'], 5: ['201807_test_잔액정보.parquet', '201808_test_잔액정보.parquet', '201809_test_잔액정보.parquet', '201810_test_잔액정보.parquet', '201811_test_잔액정보.parquet', '201812_test_잔액정보.parquet'], 6: ['201807_test_채널정보.parquet', '201808_test_채널정보.parquet', '201809_test_채널정보.parquet', '201810_test_채널정보.parquet', '201

In [14]:
# 파일 딕셔너리 생성
test_List = {}
temp = []
i = 0
# 파일 리스트에 각 파일의 데이터 프레임 저장
for files in filePathList.values():
    for file in files:
        temp.append(pd.read_parquet(file, engine='fastparquet'))
    i = i+1
    test_List[i] = temp
    temp = []

### 파일 딕셔너리를 생성했으며, 이 파일 딕셔너리에는 모든 파일의 리스트가 담겨있다.

- 데이터 프레임을 불러올 때 형식
    - **df_List[i][j]**
    - i : 폴더의 번호
    - j : 파일의 번호. (201807 : 0, 201812 : 6)
- **주의 사항 : 메모리가 작을시 시도해선 안됨**

In [ ]:
train_List[1][0].head()

,기준년월,ID,남녀구분코드,연령,Segment,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,...,할인금액_제휴연회비_B0M,청구금액_기본연회비_B0M,청구금액_제휴연회비_B0M,상품관련면제카드수_B0M,임직원면제카드수_B0M,우수회원면제카드수_B0M,기타면제카드수_B0M,카드신청건수,Life_Stage,최종카드발급경과월
0,201807,TRAIN_000000,2,40대,D,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(2),22
1,201807,TRAIN_000001,1,30대,E,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),18
2,201807,TRAIN_000002,1,30대,C,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀출산기,20
3,201807,TRAIN_000003,2,40대,D,1,1,0,1,2,...,0,0,0,0개,0개,0개,0개,1,자녀성장(2),17
4,201807,TRAIN_000004,2,40대,E,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,1,자녀성장(1),15


In [28]:
test_List[1][0].head()

,기준년월,ID,남녀구분코드,연령,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,소지카드수_이용가능_신용,...,할인금액_제휴연회비_B0M,청구금액_기본연회비_B0M,청구금액_제휴연회비_B0M,상품관련면제카드수_B0M,임직원면제카드수_B0M,우수회원면제카드수_B0M,기타면제카드수_B0M,카드신청건수,Life_Stage,최종카드발급경과월
0,201807,TEST_00000,1,40대,1,1,0,1,2,2,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),45
1,201807,TEST_00001,1,60대,1,1,0,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀독립기,19
2,201807,TEST_00002,1,40대,1,1,1,1,2,2,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),5
3,201807,TEST_00003,2,40대,1,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),7
4,201807,TEST_00004,2,40대,1,0,1,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),1


In [24]:
train_List[1][0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 78 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   기준년월                400000 non-null  int64  
 1   ID                  400000 non-null  object 
 2   남녀구분코드              400000 non-null  int64  
 3   연령                  400000 non-null  object 
 4   Segment             400000 non-null  object 
 5   회원여부_이용가능           400000 non-null  int64  
 6   회원여부_이용가능_CA        400000 non-null  int64  
 7   회원여부_이용가능_카드론       400000 non-null  int64  
 8   소지여부_신용             400000 non-null  int64  
 9   소지카드수_유효_신용         400000 non-null  int64  
 10  소지카드수_이용가능_신용       400000 non-null  int64  
 11  입회일자_신용             400000 non-null  int64  
 12  입회경과개월수_신용          400000 non-null  int64  
 13  회원여부_연체             400000 non-null  int64  
 14  이용거절여부_카드론          400000 non-null  int64  
 15  동의여부_한도증액안내         400000 non-nul

In [22]:
test_List[1][0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 77 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   기준년월                100000 non-null  int64  
 1   ID                  100000 non-null  object 
 2   남녀구분코드              100000 non-null  int64  
 3   연령                  100000 non-null  object 
 4   회원여부_이용가능           100000 non-null  int64  
 5   회원여부_이용가능_CA        100000 non-null  int64  
 6   회원여부_이용가능_카드론       100000 non-null  int64  
 7   소지여부_신용             100000 non-null  int64  
 8   소지카드수_유효_신용         100000 non-null  int64  
 9   소지카드수_이용가능_신용       100000 non-null  int64  
 10  입회일자_신용             100000 non-null  int64  
 11  입회경과개월수_신용          100000 non-null  int64  
 12  회원여부_연체             100000 non-null  int64  
 13  이용거절여부_카드론          100000 non-null  int64  
 14  동의여부_한도증액안내         100000 non-null  int64  
 15  수신거부여부_TM           100000 non-null

### 가입통신회사코드 컬럼에 결측치가 16,880개 있음을 확인 가능